<a href="https://colab.research.google.com/github/shizoda/education/blob/main/info_theory/communication_sender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 通信シミュレーション（送信側）

情報理論における符号化を学ぶため，雑音のある通信路を経由してメッセージを送信してみましょう．

## 準備

ストレージをマウントします．通信路として使用します．

In [36]:
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running on Google Colab")
    drive.mount('/content/drive')
    saved_dir = '/content/drive/MyDrive/info-theory'
except ImportError:
    IN_COLAB = False
    print("Not running on Google Colab")
    saved_dir = '//nas/info-theory'

import os
os.makedirs(saved_dir, exist_ok=True)
print(f"Saved directory is set to: {saved_dir}")
print("current directory", os.getcwd())

import sys
sys.path.append("/content/drive/MyDrive")

# GitHubリポジトリのURL
!wget -q https://raw.githubusercontent.com/shizoda/education/main/info_theory/communication_lib.py

# 必要なPythonファイルをインポートする
from communication_lib import *

Running on Google Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved directory is set to: /content/drive/MyDrive/info-theory
current directory /content


## メッセージ

送り主，宛先，メッセージ本文を指定します．

なお，サンプルの英文は "e"，和文は「静」や「岡」が多用された文になっています．

In [37]:
message_from = "alice"
message_to   = "bob"

message = "Everyone expects excellent experiences every evening. Each evening, Ellie eagerly eats exquisite edibles, enjoying every essence. Evenings envelop Ellie, embracing endless energy. Ellie engages effortlessly, expressing emotions endlessly. Every encounter elevates Ellie's enthusiasm, enriching every experience eternally."
# message = "静岡県は静かで美しい場所です。静岡市は県の中心です。静岡には静岡茶や静岡おでんなどの名物があります。静かな山々や岡の風景も楽しめます。"
print(f"メッセージ: {message}")

メッセージ: Everyone expects excellent experiences every evening. Each evening, Ellie eagerly eats exquisite edibles, enjoying every essence. Evenings envelop Ellie, embracing endless energy. Ellie engages effortlessly, expressing emotions endlessly. Every encounter elevates Ellie's enthusiasm, enriching every experience eternally.


In [38]:
# 正しく並べ替えてください
#### ここから

print("△△△符号化器：ハフマン符号")
encoded_text, codebook, padding_length = huffman_encoding(message)

△△△符号化器：ハフマン符号

Character Codes: E:69 v:118 e:101 r:114 y:121 o:111 n:110 e:101  :32 e:101 x:120 p:112 e:101 c:99 t:116 s:115  :32 e:101 x:120 c:99 e:101 l:108 l:108 e:101 n:110 t:116  :32 e:101 x:120 p:112 e:101 r:114 i:105 e:101 n:110 c:99 e:101 s:115  :32 e:101 v:118 e:101 r:114 y:121  :32 e:101 v:118 e:101 n:110 i:105 n:110 g:103 .:46  :32 E:69 a:97 c:99 h:104  :32 e:101 v:118 e:101 n:110 i:105 n:110 g:103 ,:44  :32 E:69 l:108 l:108 i:105 e:101  :32 e:101 a:97 g:103 e:101 r:114 l:108 y:121  :32 e:101 a:97 t:116 s:115  :32 e:101 x:120 q:113 u:117 i:105 s:115 i:105 t:116 e:101  :32 e:101 d:100 i:105 b:98 l:108 e:101 s:115 ,:44  :32 e:101 n:110 j:106 o:111 y:121 i:105 n:110 g:103  :32 e:101 v:118 e:101 r:114 y:121  :32 e:101 s:115 s:115 e:101 n:110 c:99 e:101 .:46  :32 E:69 v:118 e:101 n:110 i:105 n:110 g:103 s:115  :32 e:101 n:110 v:118 e:101 l:108 o:111 p:112  :32 E:69 l:108 l:108 i:105 e:101 ,:44  :32 e:101 m:109 b:98 r:114 a:97 c:99 i:105 n:110 g:103  :32 e:101 n:110 d:100 l:108 e

In [39]:

print("通信路符号化器：ハミング符号")
encoded_data = hamming_encode(encoded_text)

通信路符号化器：ハミング符号
Hamming Input  : 0100 0100 1000 1111 1110 0011 1100 1110 0001 1001 1011 1100 01.. .
Hamming Encoded: 1001100 1001100 1110000 1111111 0010110 1000011 0111100 0...


In [40]:
# 相手に届くビットには雑音が載る（一部の 0 / 1 がひっくり返る）
print("雑音のある通信路から送信")
send_message(saved_dir, message_from, message_to, encoded_data, error_rate=0.2)
#### ここまで

雑音のある通信路から送信

 △△△符号化器：ハフマン符号

Character Codes: 1:49 0:48 0:48 1:49 1:49 0:48 0:48 1:49 0:48 0:48 1:49 1:49 0:48 0:48 1:49 1:49 1:49 0:48 0:48 0:48 0:48 1:49 1:49 1:49 1:49 1:49 1:49 1:49 0:48 0:48 1:49 0:48 1:49 1:49 0:48 1:49 0:48 0:48 0:48 0:48 1:49 1:49 0:48 1:49 1:49 1:49 1:49 0:48 0:48 0:48 0:48 1:49 0:48 1:49 1:49 0:48 1:49 1:49 0:48 1:49 0:48 0:48 1:49 0:48 0:48 1:49 1:49 0:48 0:48 1:49 0:48 1:49 1:49 0:48 0:48 1:49 1:49 0:48 1:49 1:49 1:49 1:49 0:48 0:48 1:49 0:48 0:48 1:49 1:49 0:48 0:48 1:49 0:48 0:48 1:49 1:49 0:48 0:48 0:48 1:49 0:48 1:49 0:48 1:49 0:48 0:48 0:48 0:48 1:49 1:49 1:49 1:49 1:49 1:49 0:48 0:48 1:49 1:49 0:48 0:48 1:49 1:49 1:49 1:49 0:48 0:48 1:49 0:48 1:49 0:48 1:49 0:48 1:49 0:48 0:48 1:49 0:48 1:49 1:49 0:48 0:48 0:48 0:48 0:48 0:48 0:48 0:48 0:48 1:49 0:48 0:48 1:49 0:48 1:49 0:48 1:49 0:48 0:48 1:49 0:48 1:49 1:49 1:49 0:48 1:49 0:48 0:48 1:49 1:49 0:48 1:49 0:48 1:49 0:48 1:49 1:49 0:48 0:48 0:48 0:48 1:49 1:49 1:49 1:49 0:48 0:48 1:49 1:49 0:48 1:49 1: